In [1]:
import tensorflow as tf
print(f"Tensorflow version : {tf.__version__}")

Tensorflow version : 2.2.0-rc1


In [2]:
from tensorflow.python.ops import sparse_ops
import tensorflow.keras.backend as K
import pandas as pd
# import matplotlib.pyplot as plt
import os
# assert tf.test.is_gpu_available()
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import numpy as np
from tableauColors import palette

[]
Welcome to JupyROOT 6.18/00


In [3]:
from ipynb.fs.full.clustering import make_clusters, extract_features, tf_serialize_example

Tensorflow version : 2.2.0-rc1


2020-07-29 14:10:41.109589: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-07-29 14:10:41.201908: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3000000000 Hz
2020-07-29 14:10:41.211414: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x8f53870 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-07-29 14:10:41.211439: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


## Parser

In [4]:
feature_description = {
    'eventId': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([3], tf.string, default_value=["",]*3)
}

feature_shape = {
    'eventId': tf.TensorShape([1,]),
    'image': tf.TensorShape([568, 568, 2])
}

def parser_fn(proto):
    serialized = tf.io.parse_single_example(proto, feature_description)
    deserialized = {k: (tf.sparse.to_dense(sparse_ops.deserialize_sparse(v, K.floatx()))
                        if k != 'eventId' else v) for k,v in serialized.items()}
    [deserialized[k].set_shape(feature_shape[k]) for k in deserialized.keys()]
    x = deserialized['image']
    x = tf.expand_dims(x, axis=0)
    x = K.pool2d(x, pool_size=(2, 2), strides=(2, 2), pool_mode='avg')
    # sum instead of avg
    x = 4.*x
    deserialized['image'] = tf.squeeze(x)
    deserialized['S_image'] = deserialized['image'][:,:,0]
    deserialized['C_image'] = deserialized['image'][:,:,1]
    
    deserialized['tseed'] = tf.constant(4., tf.float32)
    deserialized['tneighbour'] = tf.constant(2., tf.float32)
    deserialized['tcell'] = tf.constant(0., tf.float32)
    deserialized['tenergy'] = tf.constant(1000., tf.float32)
    deserialized['tlocmax'] = tf.constant(500., tf.float32)
    deserialized['tnum'] = tf.constant(3, tf.int32)
    
    return deserialized

In [5]:
DATA_DIR = '/groups/hep/mojen/repositories/DREAM/Run/final_run/tauolaevts/25000/wothreshold'
# DATA_DIR = '/home/jupyter/DREAM'
BATCH_SIZE = 32
BUFFER_SIZE = 64

filename = [os.path.join(DATA_DIR, 'filtered.tfrecord')]
dataset = tf.data.TFRecordDataset(filename, compression_type='GZIP', buffer_size=BUFFER_SIZE)
dataset = dataset.take(20).map(parser_fn)
clusters_dataset = dataset.map(make_clusters)
features_dataset = clusters_dataset.map(extract_features)
S_serialized_features_dataset = features_dataset.map(lambda x: tf_serialize_example(x, 'S'))
C_serialized_features_dataset = features_dataset.map(lambda x: tf_serialize_example(x, 'C'))

W0729 14:10:51.608391 139991668160320 ag_logging.py:146] AutoGraph could not transform <function make_clusters at 0x7f50080df8c8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute 'add_ordinary_node'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute 'add_ordinary_node'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


W0729 14:10:53.893772 139991668160320 ag_logging.py:146] AutoGraph could not transform <function extract_features at 0x7f50080df950> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute 'add_ordinary_node'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute 'add_ordinary_node'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


W0729 14:10:53.984041 139991668160320 ag_logging.py:146] AutoGraph could not transform <function tf_serialize_example at 0x7f50080dfbf8> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute 'add_ordinary_node'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute 'add_ordinary_node'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [15]:
# list(dataset.map(lambda x: tf.reduce_max(x['S_image'])).as_numpy_iterator())

In [10]:
# import datetime
# print(datetime.datetime.now())

2020-05-29 10:40:46.817647


In [11]:
# writer = tf.data.experimental.TFRecordWriter(os.path.join(DATA_DIR, 'S_cluster.tfrecord'))
# writer.write(S_serialized_features_dataset)

In [12]:
# import datetime
# print(datetime.datetime.now())

2020-05-29 10:40:58.715627


In [8]:
# writer = tf.data.experimental.TFRecordWriter(os.path.join(DATA_DIR, 'C_cluster.tfrecord'))
# writer.write(C_serialized_features_dataset)

In [9]:
# datetime.datetime.now()

datetime.datetime(2020, 5, 25, 15, 34, 49, 530903)

In [7]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(4):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

DataLossError: corrupted record at 0

In [8]:
import ROOT
from array import array

# palette_lst = [v for k,v in palette.items() if k != 'blue']
# palette_lst = [None, None]
palette_lst = [palette['red'], palette['orange'], palette['green'], 
               palette['cyan'], palette['pink'], palette['purple']]
# palette_lst[1] = [palette['orange'], palette['red'], palette['green'], 
#                palette['cyan'], palette['pink'], palette['purple']]
ztitle = {'S': 'Scintillation signal [GeV]', 
          'C': '#check{C}erenkov signal [GeV]'}

ratio = 1.2
ratio2 = 1.2*ratio
linespacing = 0.05
linewidth = 2
title = ['Scintillator', '#check{C}erenkov']

calo_offset = -852/2+0.5
htot = [None, None]
t3 = [None, None]

titlesize = [1.1, 0.9]
offset = [0., 0.005]

def plot_output(output, flag, label):
    h = ROOT.TH2F("","", 284, -426., 426., 284, -426., 426.)
    c = ROOT.TCanvas("c1", "c1", 1300, 600)
    c.Divide(2)
    ROOT.gStyle.SetOptStat(0)
    for l,X in enumerate(['S', 'C']):
        c.cd(l+1)
        
        ROOT.gPad.SetLeftMargin(0.12*75/60)
        ROOT.gPad.SetRightMargin(0.18*75/60)
        if l == 1:
            ROOT.gPad.SetLeftMargin(0.14*75/60)
            ROOT.gPad.SetRightMargin(0.16*75/60)
        ROOT.gPad.SetBottomMargin(0.15)
        ROOT.gPad.SetTopMargin(0.15)
        
        
        X_image = output[X+'_image']
        indices = tf.where(X_image)
        values = tf.reshape(tf.gather_nd(X_image, indices), [-1,1])
        X_image = tf.concat([tf.cast(indices, values.dtype), values], axis=1)

        htot[l] = h.Clone()
        for i,j,x in X_image:
            htot[l].Fill(3*j+calo_offset, 3*i+calo_offset, x)
        htot[l].GetZaxis().SetTitle(ztitle[X]) # + f" / {htot[l].GetXaxis().GetBinWidth(1):.1g} mm #times {htot[l].GetYaxis().GetBinWidth(1):.1g} mm")
        htot[l].GetXaxis().SetTitle('X coordinate [mm]')
        htot[l].GetYaxis().SetTitle('Y coordinate [mm]')
        
        htot[l].GetXaxis().SetTitleSize(ratio2 * htot[l].GetXaxis().GetTitleSize())
        htot[l].GetYaxis().SetTitleSize(ratio2 * htot[l].GetYaxis().GetTitleSize())
        htot[l].GetZaxis().SetTitleSize(ratio2 * htot[l].GetZaxis().GetTitleSize())

        labelsize = ratio * htot[l].GetXaxis().GetLabelSize()
        htot[l].GetXaxis().SetLabelSize(labelsize)
        htot[l].GetYaxis().SetLabelSize(labelsize)
        htot[l].GetZaxis().SetLabelSize(labelsize)

        labeloffset = (ratio + 1) * htot[l].GetXaxis().GetLabelOffset()
        htot[l].GetXaxis().SetLabelOffset(labeloffset)
        htot[l].GetYaxis().SetLabelOffset(labeloffset)
        # h[i].GetZaxis().SetLabelOffset(labeloffset)

        htot[l].GetXaxis().SetTitleOffset(ratio2 * htot[l].GetXaxis().GetTitleOffset())
        htot[l].GetYaxis().SetTitleOffset(1.12 * htot[l].GetXaxis().GetTitleOffset())
        htot[l].GetZaxis().SetTitleOffset(1. * htot[l].GetXaxis().GetTitleOffset())

        htot[l].GetXaxis().SetTickLength(0.7 * ratio * htot[l].GetXaxis().GetTickLength())
        htot[l].GetYaxis().SetTickLength(0.7 * ratio * htot[l].GetYaxis().GetTickLength())
        htot[l].GetZaxis().SetTickLength(0.7 * ratio * htot[l].GetZaxis().GetTickLength())
        
        htot[l].GetXaxis().CenterTitle()
        htot[l].GetYaxis().CenterTitle()
        htot[l].GetZaxis().CenterTitle()
        
#         htot[l].GetXaxis().SetTitleOffset(htot[l].GetXaxis().GetTitleOffset() * ratio)
#         htot[l].GetYaxis().SetTitleOffset(htot[l].GetYaxis().GetTitleOffset() * ratio)
#         htot[l].GetZaxis().SetTitleOffset(htot[l].GetZaxis().GetTitleOffset() * (ratio + 0.6))

#         htot[l].GetXaxis().SetTitleSize(htot[l].GetXaxis().GetTitleSize() * (ratio + 0.2))
#         htot[l].GetYaxis().SetTitleSize(htot[l].GetYaxis().GetTitleSize() * (ratio + 0.2))
#         htot[l].GetZaxis().SetTitleSize(htot[l].GetZaxis().GetTitleSize() * (ratio + 0.2))
        
#         htot[l].GetXaxis().SetLabelSize(htot[l].GetXaxis().GetLabelSize() * ratio)
#         htot[l].GetYaxis().SetLabelSize(htot[l].GetYaxis().GetLabelSize() * ratio)
#         htot[l].GetZaxis().SetLabelSize(htot[l].GetZaxis().GetLabelSize() * ratio)
        
#         htot[l].GetXaxis().SetTickSize(htot[l].GetXaxis().GetTickLength() * ratio)
#         htot[l].GetYaxis().SetTickSize(htot[l].GetYaxis().GetTickLength() * ratio)
#         htot[l].GetZaxis().SetTickSize(htot[l].GetZaxis().GetTickLength() * ratio)
        
#         htot[l].GetXaxis().SetLabelOffset(htot[l].GetXaxis().GetLabelOffset() * ratio)
#         htot[l].GetYaxis().SetLabelOffset(htot[l].GetYaxis().GetLabelOffset() * ratio)
#         htot[l].GetZaxis().SetLabelOffset(htot[l].GetZaxis().GetLabelOffset() * ratio)
        
#         htot[l].SetAxisRange(50., 200., 'X')
#         htot[l].SetAxisRange(50., 200., 'Y')
#         htot[l].SetAxisRange(0., 1200., 'Z')
#         htot[l].SetAxisRange(-95., 95., 'X')
#         htot[l].SetAxisRange(-95., 95., 'Y')
#         htot[l].SetAxisRange(0., 1200., 'Z')
        
        htot[l].GetYaxis().SetNdivisions(504)
        htot[l].GetXaxis().SetNdivisions(504)
        
        htot[l].Draw("COLZ")
#         htot[l].GetXaxis().ChangeLabel(1,-1,-1,11)
#         htot[l].GetYaxis().ChangeLabel(1,-1,-1,31)

        X_cluster = output[X+'_cluster'].numpy()
        if X_cluster.size != 0:
            hk = []
            u = np.unique(X_cluster[:,0], axis=0).astype(int)
            arr = array('d',[0.5])
            lin_idx = np.zeros(len(u))
            for k in range(len(u)):
                for i,j,x in X_cluster[X_cluster[...,0]==u[k]][:,1:]:
                    hk.append(h.Clone())
                    hk[k].Fill(3*j+calo_offset, 3*i+calo_offset, x)
                lin_idx[k] = (hk[k].GetMean(1)-calo_offset)*852+hk[k].GetMean(2)-calo_offset
            
#             print(np.argsort(lin_idx))
#             for k in range(len(u)):
            for m,k in enumerate(np.argsort(lin_idx)):
                hk[k].SetLineColor(palette_lst[k].GetNumber())
                hk[k].SetLineWidth(linewidth)
                hk[k].SetContour(1, arr)
                hk[k].DrawCopy("cont3 list same")
                
        if flag:
            htot[l].SetTitle(title[l])
            ROOT.gStyle.SetTitleFont(62, 't')
            ROOT.gStyle.SetTitleFontSize(titlesize[l] * ratio * htot[l].GetXaxis().GetTitleSize())
            ROOT.gStyle.SetTitleX(ROOT.gPad.GetLeftMargin()+(1-ROOT.gPad.GetLeftMargin()-ROOT.gPad.GetRightMargin())/2)
            ROOT.gStyle.SetTitleY(1-ROOT.gPad.GetTopMargin()+htot[l].GetXaxis().GetTickLength()+offset[l])
            ROOT.gStyle.SetTitleAlign(ROOT.kHAlignCenter+ROOT.kVAlignBottom)
        
        t3[l] = ROOT.TLatex(1-ROOT.gPad.GetRightMargin()-2*htot[l].GetYaxis().GetTickLength()+0.01, 
                            1-ROOT.gPad.GetTopMargin()-2*htot[l].GetXaxis().GetTickLength(), label)
        t3[l].SetTextSize(2*labelsize)
        t3[l].SetTextFont(42)
        t3[l].SetTextAlign(ROOT.kHAlignRight+ROOT.kVAlignTop)
        t3[l].SetNDC(ROOT.kTRUE)
        t3[l].Draw()
        
        c.Modified()
        c.Update()
                
    c.SaveAs("cluster_plots/cluster"+str(output['eventId'][0].numpy())+"_"+str(flag)+".png")

In [12]:
label = {"e": [7, 14], '#mu': [0, 2], '#pi': [3, 10], 
         "#rho": [15, 20], "a_{1}": [1, 4]}

for i,output in enumerate(clusters_dataset.take(21)):
    for k,v in label.items():
        if output['eventId'][0].numpy() in v:
            for j in [True, False]:
                plot_output(output, j, k)

Info in <TCanvas::Print>: png file cluster_plots/cluster0_True.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster0_False.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster1_True.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster1_False.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster2_True.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster2_False.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster3_True.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster3_False.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster4_True.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster4_False.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster7_True.png has been created
Info in <TCanvas::Print>: png file cluster_plots/cluster7_False.png has

In [11]:
df = pd.read_csv(os.path.join(DATA_DIR, 'truth.csv'))

In [119]:
df

,eventId,PrimaryDecayMode,showerId,VecShowerPDG,IsCharged,VecShowerEnergy,VecShowerScntCoMi,VecShowerScntCoMj,VecShowerScntRad,VecShowerCkovCoMi,VecShowerCkovCoMj,VecShowerCkovRad
0,0,1,1,13,1,19816.199882,377.273702,234.532187,20.828887,372.314741,237.545817,22.777741
1,1,4,3,-211,1,13397.354239,254.634528,250.203347,44.876718,258.966304,249.183696,35.940082
2,1,4,4,22,0,10115.048809,271.690628,361.068118,7.540858,270.860509,360.469369,6.590106
3,1,4,5,22,0,4740.947309,303.162344,351.590579,8.119874,303.226721,351.388664,6.192851
4,1,4,6,22,0,11594.263998,276.520498,313.499273,7.501287,276.073295,313.960087,5.838985
5,1,4,7,22,0,2466.543425,317.300468,294.328310,9.751201,316.278027,295.950673,5.310861
6,2,1,10,13,1,3521.013148,253.672810,520.511670,22.288263,253.475000,519.212500,24.868432
7,3,2,12,-211,1,17549.797631,380.544308,265.233744,50.591990,383.089440,263.093750,34.327259
8,4,4,14,-211,1,6158.922797,126.658385,283.381544,35.633573,112.870588,279.094118,40.512104
9,4,4,15,22,0,1374.572872,256.408567,320.371563,8.670616,253.228395,321.697531,5.384962


In [97]:
for i in range(5):
    print(df.loc[df.PrimaryDecayMode == i].head(10).eventId.unique())

[ 7 14 19 21 27 29 32 35 46 52]
[ 0  2  8  9 12 13 16 17 33 37]
[ 3 10 31 55 57 58 59 65 69 81]
[15 20 23 24]
[1 4]


In [120]:
df.loc[df.PrimaryDecayMode == 3].head(10)

,eventId,PrimaryDecayMode,showerId,VecShowerPDG,IsCharged,VecShowerEnergy,VecShowerScntCoMi,VecShowerScntCoMj,VecShowerScntRad,VecShowerCkovCoMi,VecShowerCkovCoMj,VecShowerCkovRad
30,15,3,51,22,0,1340.003926,210.540223,316.220916,10.501049,211.007519,317.330827,5.744330
31,15,3,52,22,0,749.435141,418.847579,211.385128,6.714958,418.030000,212.810000,4.846651
32,15,3,53,-211,1,35597.061677,244.575267,297.008262,16.512331,242.869524,295.180901,13.685899
41,20,3,70,22,0,14152.000247,210.663483,334.167984,8.430864,210.242206,333.250600,6.577069
42,20,3,71,22,0,1318.119921,188.684181,282.946895,8.236401,187.050420,281.411765,7.133742
43,20,3,72,-211,1,14440.365318,266.816361,249.178720,62.086184,248.172857,227.332857,59.332405
50,23,3,83,22,0,5512.665587,318.863469,305.249112,7.124472,319.057895,304.822807,5.553139
51,23,3,84,22,0,254.974353,400.544183,130.058487,6.406898,400.125000,129.500000,3.129831
52,23,3,85,-211,1,35702.642830,246.288243,298.380982,48.624719,239.426516,298.686536,36.476351
53,24,3,87,22,0,15000.364635,277.539280,318.262113,8.668463,278.026442,318.843750,7.610895
